In [8]:
import os
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
US_BUSINESS_DAY = CustomBusinessDay(calendar=USFederalHolidayCalendar())
from os.path import exists
import pandas as pd
from backtesting import Strategy, Backtest, set_bokeh_output
import pandas_ta as ta
# from ta_utils import *
import math

In [9]:


DATA_PATH = "."


def range_size(df: pd.DataFrame, range_period: float, range_multiplier: int):
    wper = range_period * 2 - 1
    # last candle is last index, not 0
    average_range = ta.ema(df.diff().abs(), range_period)
    AC = ta.ema(average_range, wper) * range_multiplier
    return AC


def range_filter(x: pd.DataFrame, r: pd.DataFrame) -> pd.DataFrame:
    range_filt = x.copy()
    hi_band = x.copy()
    lo_band = x.copy()

    for i in range(x.size):
        if i < 1:
            continue
        if math.isnan(r[i]):
            continue
        if x[i] > nz(range_filt[i - 1]):
            if x[i] - r[i] < nz(range_filt[i - 1]):
                range_filt[i] = nz(range_filt[i - 1])
            else:
                range_filt[i] = x[i] - r[i]
        else:
            if x[i] + r[i] > nz(range_filt[i - 1]):
                range_filt[i] = nz(range_filt[i - 1])
            else:
                range_filt[i] = x[i] + r[i]

        hi_band[i] = range_filt[i] + r[i]
        lo_band[i] = range_filt[i] - r[i]

    return hi_band, lo_band, range_filt


def nz(x) -> float:
    res = x
    if math.isnan(x):
        res = 0.0

    return res


def vumanchu_swing(df, swing_period, swing_multiplier):
    smrng = range_size(df['Close'], swing_period, swing_multiplier)
    hi_band, lo_band, range_filt = range_filter(df['Close'], smrng)
    return range_filt


class MixedStrategy(Strategy):
    ema_period_1 = 50
    ema_period_2 = 200
    ema_lookback = 50

    swing_period = 20
    swing_multiplier = 3.5

    stop_loss_percent = 15

    last_purchase_price = 0
    long_hold = 0
    short_hold = 0
    i = 0

    def init(self):
        super().init()

        #  Calculate indicators
        self.range_filter = self.I(vumanchu_swing, self.data.df, self.swing_period, self.swing_multiplier)

    def next(self):
        super().init()

        self.i += 1

        long_entry_signals = 0
        long_exit_signals = 0
        short_entry_signals = 0
        short_exit_signals = 0

        #  Check Vuman Chu Range Filter
        if self.range_filter[-1] > self.range_filter[-2]:
            long_entry_signals += 1
            short_exit_signals += 1

        if self.range_filter[-1] < self.range_filter[-2]:
            long_exit_signals += 1
            short_entry_signals += 1

        #  Stop loss
        price = self.data.df['Close'][-1]
        if price <= self.last_purchase_price * (1 - self.stop_loss_percent/100):
            long_exit_signals += 1

        if price >= self.last_purchase_price * (1 + self.stop_loss_percent/100):
            short_exit_signals += 1

        #  LONG
        #--------------------------------------------------
        if self.long_hold == 0 and long_entry_signals >= 1:
            #  Buy
            self.long_hold = 1
            self.position.close()
            self.buy()
            self.last_purchase_price = price

        elif self.long_hold == 1 and long_exit_signals >= 1:
            # Close any existing long trades, and sell the asset
            self.long_hold = 0
            self.position.close()
            self.last_purchase_price = 0

        #  SHORT
        #--------------------------------------------------
        if self.short_hold == 0 and short_entry_signals >= 1:
            #  Sell
            self.short_hold = 1
            self.position.close()
            self.sell()
            self.last_purchase_price = price

        elif self.short_hold == 1 and short_exit_signals >= 1:
            # Close any existing long trades, and sell the asset
            self.short_hold = 0
            self.position.close()
            self.last_purchase_price = 0

def load_data_file(symbol, interval, time_interval):

    file_path = os.path.join(DATA_PATH, f"{symbol}_{interval}_{time_interval}.csv")
    if not exists(file_path):
        print(f"Error loading file {file_path}")
        return None

    #  Load data file
    df = pd.read_csv(file_path, index_col='Date', parse_dates=True)
    if df is None or len(df) == 0:
        print(f"Empty file: {file_path}")
        return None

    #  Shorten df for testing
    #df = df.iloc[30000:40000]

    #  Rename columns
    # print(df.head())
    df = df.rename(columns={"Open": "Open", "Close": "Close", "Low": "Low", "High": "High", "Volume": "Volume"})

    # df['Date'] = pd.to_datetime(df['unix'], unit='s', utc=True)
    df = df.drop(['Adj Close'], axis=1)

    # print(df.head())

#     df = df.set_index('Date')

    #  Drop na
    df.dropna(axis=0, how='any', inplace=True)

    return df


def run_backtest(df):
    # If exclusive orders (each new order auto-closes previous orders/position),
    # cancel all non-contingent orders and close all open trades beforehand
    bt = Backtest(df, MixedStrategy, cash=100000, commission=.00075, trade_on_close=True, exclusive_orders=False, hedging=False)
    stats = bt.run()
    print(stats)
#     set_bokeh_output(notebook=False)

    bt.plot()

In [10]:
df = load_data_file('BTC-USD', '1h', '6d')

In [11]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2023-02-25 00:00:00+00:00,23197.087891,23210.210938,23152.423828,23165.320312,0
2023-02-25 01:00:00+00:00,23167.687500,23206.890625,23159.476562,23204.515625,0
2023-02-25 02:00:00+00:00,23199.054688,23207.302734,23098.121094,23104.314453,0
2023-02-25 03:00:00+00:00,23106.859375,23128.851562,23093.617188,23108.359375,239616
2023-02-25 04:00:00+00:00,23109.886719,23122.630859,23082.904297,23122.630859,19922944
...,...,...,...,...,...
2023-03-03 05:00:00+00:00,22400.986328,22424.228516,22362.820312,22367.246094,302645248
2023-03-03 06:00:00+00:00,22367.369141,22415.994141,22360.468750,22402.166016,216111104
2023-03-03 07:00:00+00:00,22399.292969,22412.218750,22362.826172,22384.712891,187672576


In [12]:
run_backtest(df)

Start                     2023-02-25 00:00...
End                       2023-03-03 09:00...
Duration                      6 days 09:00:00
Exposure Time [%]                   98.701299
Equity Final [$]                 98621.489529
Equity Peak [$]                      100000.0
Return [%]                           -1.37851
Buy & Hold Return [%]               -3.300141
Return (Ann.) [%]                   27.439347
Volatility (Ann.) [%]               70.648774
Sharpe Ratio                         0.388391
Sortino Ratio                        0.930059
Calmar Ratio                         4.858595
Max. Drawdown [%]                   -5.647589
Avg. Drawdown [%]                   -5.647589
Max. Drawdown Duration        6 days 08:00:00
Avg. Drawdown Duration        6 days 08:00:00
# Trades                                   22
Win Rate [%]                        27.272727
Best Trade [%]                       4.055662
Worst Trade [%]                     -1.865274
Avg. Trade [%]                    